In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
import logging

In [2]:
logging.basicConfig(level=logging.INFO, format=' %(asctime)s - %(levelname)s - %(message)s')

In [3]:
# 屏幕最大化，且指定下载目录
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")

prefs = {"profile.default_content_settings.popups": 0,
         "directory_upgrade": True}
options.add_experimental_option("prefs", prefs)

In [4]:
# 使用chromedriver才可以用开发者权限
chrome_driver_path = ".//chrome_driver//chromedriver.exe"
browser = webdriver.Chrome(chrome_driver_path, chrome_options=options)

In [5]:
# main page
main_page_url = "http://shopee.sg"
browser.get(main_page_url)

# remove ads, refresh again
browser.get(main_page_url)
time.sleep(10)
browser.get(main_page_url)
time.sleep(10)
browser.get(main_page_url)
time.sleep(10)

In [6]:
# login
LoginElem = browser.find_elements_by_css_selector('.navbar__link.navbar__link--account.navbar__link--tappable.navbar__link--hoverable.navbar__link-text.navbar__link-text--medium')
'''
有关Selenium Compound class names not permitted的错误
可以参考https://stackoverflow.com/questions/37771604/selenium-compound-class-names-not-permitted
使用css_selector解决即可
'''
# 点击登入，弹出账户密码输入框
LoginElem[1].click()
time.sleep(10)

In [7]:
acc_password_input_elem = browser.find_elements_by_css_selector('._2QBp41._1b-IZR')
try:
    acc_password_input_elem[0].click()
    acc_password_input_elem[0].send_keys('tengus1.sg')
    acc_password_input_elem[1].click()
    acc_password_input_elem[1].send_keys('tengus1803')
except:
    time.sleep(30)
    acc_password_input_elem = browser.find_elements_by_css_selector('._2QBp41._1b-IZR')
    acc_password_input_elem[0].click()
    acc_password_input_elem[0].send_keys('tengus1.sg')
    acc_password_input_elem[1].click()
    acc_password_input_elem[1].send_keys('tengus1803')
time.sleep(10)

In [8]:
Login_button_elem = browser.find_elements_by_css_selector('._2DvX7K._3j9-lD._3ddytl.SjORHu')
time.sleep(10)
Login_button_elem[0].click()
time.sleep(10)

In [9]:
# change current tab size
# browser.set_window_size(400, 862)    

In [10]:
# add fans from Top Seller's shop
top_shop_id = 11918
top_shop_url = 'https://shopee.sg/shop/' + str(top_shop_id) + '/followers/?__classic__=1'
browser.get(top_shop_url)

In [11]:
# add certain number of fans
to_add_num_of_following = 400
num_of_following_display = len(browser.find_elements_by_css_selector('.clickable_area.middle-centered-div'))

In [12]:
# page down until we get at least 400 fans to cancel
# while num_of_following_display <= 1000:
    # body = browser.find_element_by_css_selector('body')
    # body.send_keys(Keys.PAGE_DOWN)
    # body.send_keys(Keys.END)
    # time.sleep(5)
    # num_of_following_display = len(browser.find_elements_by_css_selector('.clickable_area.middle-centered-div'))
    
# num_of_following_display

In [13]:
following_buttons = browser.find_elements_by_css_selector('.btn-follow.follow.L14')
# len(following_buttons)

In [37]:
i = 0
total_add_num_of_following = to_add_num_of_following

In [38]:
browser.find_element_by_css_selector('body').send_keys(Keys.CONTROL + Keys.HOME)

In [39]:
while i <= total_add_num_of_following - 1:  
    while to_add_num_of_following > 0:
        if i + 1 > len(following_buttons):
            browser.find_element_by_css_selector('body').send_keys(Keys.PAGE_DOWN)
            time.sleep(10)
        
        following_buttons = browser.find_elements_by_css_selector('.btn-follow.follow.L14')
        logging.info('Now we have ' + str(len(following_buttons)) + ' following buttons on the screen.')
        logging.info('Now run the ' + str(i + 1) + ' time.')
        # if following_buttons[i].text == '+ 關注':
        # if following_buttons[i].text == '+ ติดตาม':
        if following_buttons[i].text == '+ Follow':
            # print(following_buttons[i].text)
            shopid = following_buttons[i].get_attribute('shopid')
            following_buttons[i].click()
            time.sleep(5)
            to_add_num_of_following -= 1
            logging.info(str(shopid) + ' is following now, ' + str(to_add_num_of_following) + ' following remains.')
        else:
            logging.warning('Skip this one. It\'s following already.')
        i += 1

 2019-10-20 00:20:18,939 - INFO - Now we have 20 following buttons on the screen.
 2019-10-20 00:20:18,940 - INFO - Now run the 1 time.
 2019-10-20 00:20:24,050 - INFO - 189953859 is following now, 43 following remains.
 2019-10-20 00:20:24,060 - INFO - Now we have 40 following buttons on the screen.
 2019-10-20 00:20:24,061 - INFO - Now run the 2 time.
 2019-10-20 00:20:29,129 - INFO - 189953808 is following now, 42 following remains.
 2019-10-20 00:20:29,139 - INFO - Now we have 40 following buttons on the screen.
 2019-10-20 00:20:29,141 - INFO - Now run the 3 time.
 2019-10-20 00:20:34,217 - INFO - 189953800 is following now, 41 following remains.
 2019-10-20 00:20:34,238 - INFO - Now we have 40 following buttons on the screen.
 2019-10-20 00:20:34,239 - INFO - Now run the 4 time.
 2019-10-20 00:20:39,318 - INFO - 189953759 is following now, 40 following remains.
 2019-10-20 00:20:39,331 - INFO - Now we have 40 following buttons on the screen.
 2019-10-20 00:20:39,332 - INFO - Now 

 2019-10-20 00:21:10,599 - INFO - Now run the 39 time.
 2019-10-20 00:21:10,615 - WARNING - Skip this one. It's following already.
 2019-10-20 00:21:10,625 - INFO - Now we have 40 following buttons on the screen.
 2019-10-20 00:21:10,625 - INFO - Now run the 40 time.
 2019-10-20 00:21:10,641 - WARNING - Skip this one. It's following already.
 2019-10-20 00:21:20,697 - INFO - Now we have 40 following buttons on the screen.
 2019-10-20 00:21:20,697 - INFO - Now run the 41 time.


IndexError: list index out of range

In [ ]:
# 关闭
browser.quit()